# Imports

In [ ]:
#libraries
import tensorflow as tf
import time
import numpy as np
from matplotlib import pyplot as plt
import cv2
#Models

#skeleton CNN:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


#LSTM
model = tf.keras.models.load_model('lstm.h5')

Mounted at /content/drive


# Acces the camera


In [ ]:
# Access the webcam
cap = cv2.VideoCapture(0)

# Define variables
num_frames = 2  # Number of frames to capture per second
duration = 12  # Duration in seconds
total_frames = num_frames * duration

# Initialize a list to store the captured frames
frames = []

# Set start time
start_time = time.time()

# Loop until desired number of frames is captured
while len(frames) < total_frames:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret:
        # Resize frame to match the input size of the skeleton CNN model
        img = cv2.resize(frame, (256, 256))
        input_image = tf.cast(img, dtype=tf.float32)
        input_image = np.expand_dims(input_image, axis=0)

        # Set the frame as the input tensor to the skeleton CNN model
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference on the skeleton CNN model
        interpreter.invoke()

        # Get the output tensor from the skeleton CNN model
        output_data = interpreter.get_tensor(output_details[0]['index'])

        # Append the output tensor to the list of frames
        frames.append(output_data)

        # Display the frame (optional)
        cv2_imshow(frame)

    # Break the loop if the duration has exceeded
    if time.time() - start_time > duration:
        break

# Release the video capture object and close any open windows
cap.release()
cv2.destroyAllWindows()

# Convert the list of frames to a TensorFlow tensor
X_test = tf.convert_to_tensor(frames, dtype=tf.float32)

# Reshape the tensor to match the LSTM model's input shape
X_test = tf.reshape(X_test, (1, total_frames, 34))

# Perform inference using the LSTM model
predictions = model.predict(X_test)

# Print the predictions
print(predictions)